In [0]:
model = mlflow.pyfunc.load_model(model_path)

from pyspark.ml.evaluation import RegressionEvaluator

eval_rmse = RegressionEvaluator(labelCol="median_house_value", predictionCol="prediction", metricName="rmse")

rmse = eval_rmse.evaluate(predDF)

print(rmse)

registry_uri="databricks://rmr:rmr"

# Use ML Flow to register the experiments in the local registry and register the trained model in the shared registry
import mlflow
import mlflow.spark

mlflow.set_experiment("/SparkML")

with mlflow.start_run(run_name="SparkML-Test") as run:

  # Log the algorithm parameter num_trees to the run
  mlflow.log_param('num_trees', 10)


  # Train model with Training Data
  pipelineModel = pipeline.fit(trainDF)

  #Scoring the test data
  predDF = pipelineModel.transform(testDF)
  
  #Evaluate the accuracy metrics
  eval_rmse = RegressionEvaluator(labelCol="median_house_value", predictionCol="prediction", metricName="rmse")

  rmse = eval_rmse.evaluate(predDF)

  print('RMSE is', str(rmse))
  mlflow.log_param('rmse', rmse)
  

  # Log model
  print("Stage: Log Model Pipeline - Status: Started")
  mlflow.spark.log_model(pipelineModel, "model")
  print("Stage: Log Model Pipeline - Status: Complete")

  run_id = mlflow.active_run().info.run_id



# COMMAND ----------

# MAGIC %md
# MAGIC 
# MAGIC Register the model in the shared registry

# COMMAND ----------

mlflow.set_registry_uri(registry_uri)

# COMMAND ----------

#Register the mode with name: SparkModel
# run_id = run.info.run_id
# model_uri = f"runs:/{run_id}/model"
# model_details = mlflow.register_model(model_uri=model_uri, name="SparkModel")

74048.90538268468


2023/05/24 06:34:31 INFO mlflow.tracking.fluent: Experiment with name '/SparkML' does not exist. Creating a new experiment.


RMSE is 74048.90538268468
Stage: Log Model Pipeline - Status: Started


2023/05/24 06:34:43 INFO mlflow.spark: Inferring pip requirements by reloading the logged model from the databricks artifact repository, which can be time-consuming. To speed up, explicitly specify the conda_env or pip_requirements when calling log_model().
2023/05/24 06:36:46 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: dbfs:/databricks/mlflow-tracking/2648146100547158/e18ed90be12e48db9fd562e9a76d58ff/artifacts/model/sparkml, flavor: spark), fall back to return ['pyspark==3.3.2', 'pandas<2']. Set logging level to DEBUG to see the full traceback.


Stage: Log Model Pipeline - Status: Complete
